In [5]:
import boto3 

# Create a Key Pair for Secure Connections

In [6]:
ec2 = boto3.client('ec2', region_name='us-east-1') 

ec2.describe_instances() 

resp = ec2.create_key_pair(KeyName='kgptalkie')


In [8]:
file = open("creds/mlops.pem", 'w')
file.write(resp['KeyMaterial'])
file.close() 


# Check if instance is present

In [2]:
import boto3

ec2 = boto3.client('ec2', region_name='us-east-1') 


response = ec2.describe_instances()

# instance_name = "mlops-kgptalkie"

instance_name = "mlops-prod"

instance_id="" 

for resp in response['Reservations']:
    resp = resp['Instances'][0]
    tags = resp.get('Tags', [])
    
    for tag in tags:
        if tag.get("Key", "")=="Name" and tag.get("Value", "")==instance_name:
            instance_id = resp['InstanceId']

if instance_id=="":
    print(f"No instance found with name {instance_name}")
    # raise("Stop here!!!")

instance_id

No instance found with name mlops-prod


''

# Create an EC2 instance

In [9]:
import boto3

ec2 = boto3.client('ec2', region_name='us-east-1') 

if instance_id == "":
    response = ec2.run_instances(
        ImageId = 'ami-0197c13a4f68c9360',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.large',
        KeyName='kgptalkie',
        BlockDeviceMappings=[
            {
                "DeviceName": "/dev/xvda",
                'Ebs':{
                    'DeleteOnTermination': True,
                    'VolumeSize': 120
                }
            }
        ]

    )

    instance_id = response['Instances'][0]['InstanceId']

    ec2.create_tags(Resources=[instance_id], Tags=[
        {
            'Key':'Name',
            'Value':instance_name
        }
    ])

else:
    print("Instance is already present")
    

# Create a Secure Group and Rules to it

In [10]:
group_name = 'kgptalkie'

response = ec2.describe_security_groups()

security_group_id = [x['GroupId'] for x in response['SecurityGroups'] if x['GroupName']==group_name]

if security_group_id == []:
    response = ec2.create_security_group(
        GroupName = group_name,
        Description = "Security group for testing"
    )
    security_group_id = response['GroupId']
else:
    security_group_id = security_group_id[0]

security_group_id


'sg-082a2af62702b622a'

In [11]:
from botocore.exceptions import ClientError

def update_security_group(group_id, protocol, port, cidr):
    try:
        response = ec2.authorize_security_group_ingress(
            GroupId = group_id,
            IpPermissions=[
                {
                    'IpProtocol': protocol,
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': cidr}]
                }
            ]
        )
    except ClientError as e:
        if e.response['Error']['Code']=='InvalidPermission.Duplicate':
            print('This rule is already there')
        else:
            print("an error as occured!")
            print(e)

update_security_group(security_group_id, 'tcp', 22, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 80, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8501, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8502, '0.0.0.0/0')


In [12]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])


{'ResponseMetadata': {'RequestId': 'ba9aff43-85b7-4e46-9646-f54a8ecde255',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ba9aff43-85b7-4e46-9646-f54a8ecde255',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Sat, 19 Apr 2025 11:34:55 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# Attach S3 Policy to EC2 instance

In [18]:
import boto3

iam = boto3.client('iam')

response = iam.list_roles()

for role in response['Roles']:
    print(role['RoleName'])

AmazonSageMaker-ExecutionRole-20250414T194267
AmazonSageMakerBedrockModelConsumptionRole-03cbf98ad72958
AmazonSageMakerBedrockModelConsumptionRole-05b247e0485669
AmazonSageMakerBedrockModelConsumptionRole-15eed14c11e101
AmazonSageMakerBedrockModelConsumptionRole-181795e1c15d61
AmazonSageMakerBedrockModelConsumptionRole-2d1c11d7571020
AmazonSageMakerBedrockModelConsumptionRole-3d06be639af4e9
AmazonSageMakerBedrockModelConsumptionRole-4603f73e3211a1
AmazonSageMakerBedrockModelConsumptionRole-46ca82e8fe0f7c
AmazonSageMakerBedrockModelConsumptionRole-5bd685c3ba6cbf
AmazonSageMakerBedrockModelConsumptionRole-7507deb98df51a
AmazonSageMakerBedrockModelConsumptionRole-779865b97aeb74
AmazonSageMakerBedrockModelConsumptionRole-82aefb84fe1a2e
AmazonSageMakerBedrockModelConsumptionRole-82cd6a0c9f4a30
AmazonSageMakerBedrockModelConsumptionRole-86ecbf891b3fea
AmazonSageMakerBedrockModelConsumptionRole-8d514f221977ac
AmazonSageMakerBedrockModelConsumptionRole-957e834a1fdc48
AmazonSageMakerBedrockMode

In [ ]:
# describe IAM role
iam = boto3.client('iam')

role_name = "ec2-s3-full-access"

response = iam.get_role(RoleName=role_name)

role_arn = response['Role']['Arn']

role_arn

# Ensure there is an instance profile with the same name as the role
instance_profile_name = role_name
try:
    iam.get_instance_profile(InstanceProfileName=instance_profile_name)
except iam.exceptions.NoSuchEntityException:
    # Create an instance profile if it doesn't exist
    iam.create_instance_profile(InstanceProfileName=instance_profile_name)
    # Add role to the instance profile
    iam.add_role_to_instance_profile(
        InstanceProfileName=instance_profile_name,
        RoleName=role_name
    )

# Attach the instance profile to the EC2 instance
ec2.associate_iam_instance_profile(
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    InstanceId=instance_id
)


NoSuchEntityException: An error occurred (NoSuchEntity) when calling the GetRole operation: The role with name AdministratorAccess cannot be found.

# Terminate EC2 instance

In [ ]:
import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        
        time.sleep(10)

def terminate_instances(instance_id):
    print("EC2 Instance Termination")
    ec2.terminate_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'terminated')

# terminate_instances([instance_id])